<a href="https://colab.research.google.com/github/Kaiziferr/NLP_Workshop/blob/master/GNT/w0_gnt_deep_learning_chars_dummy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from urllib.request import urlopen
from urllib.parse import unquote

from os import sys

# Loada Data

In [ ]:
url = urlopen('https://gist.githubusercontent.com/jsdario/1daee22f3f13fe6bc6a343f829565759/raw/3511dc6de6a7bf064c168b4f20b85a20d8f83b91/evangelio_segun_marcos.txt')
cuento = url.readlines()
cuento=list(map(lambda x: x.decode('UTF-8'), cuento))
cuento = " ".join(cuento).lower()

'evangelio segun marcos\n jorge luis borges\n \n el hecho sucedió en la estancia los álamos, en el partido de junín, hacia el sur, en los últimos días del mes de marzo de 1928. su protagonista fue un estudiante de medicina, baltasar espinosa. podemos definirlo por ahora como uno de tantos muchachos porteños, sin otros rasgos dignos de nota que esa facultad oratoria que le había hecho merecer más de un premio en el colegio inglés de ramos mejía y que una casi ilimitada bondad. no le gustaba discutir; prefería que el interlocutor tuviera razón y no él. aunque los azares del juego le interesaban, era un mal jugador, porque le desagradaba ganar. su abierta inteligencia era perezosa; a los treinta y tres años le faltaba rendir una materia para graduarse, la que más lo atraía. su padre, que era librepensador, como todos los señores de su época, lo había instruido en la doctrina de herbert spencer, pero su madre, antes de un viaje a montevideo, le pidió que todas las noches rezara el padrenue

# Map chars

In [ ]:
chars = sorted(list(set(cuento)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

In [ ]:
n_chars = len(cuento)
n_vocab = len(chars)
print(f'Total caracteres {n_chars}')
print(f'Total vocales {n_vocab}')

Total caracteres 11206
Total vocales 43


# Input Vector

In [ ]:
seq_length = 100
X_data = []
y_data = []
for i in range(0, n_chars-seq_length):
  seq_in = cuento[i:i+seq_length]
  seq_out = cuento[i + seq_length]
  X_data.append([char_to_int[char] for char in seq_in])
  y_data.append(char_to_int[seq_out])
n_patterns = len(X_data)
print(f'Total patrones: {n_patterns}')

Total patrones: 11106


In [ ]:
#muestras, pasos de tiempo, características
X = np.reshape(X_data, (n_patterns, seq_length,1))
# Normalization
X = X/float(n_vocab)
# One hot encode
y = np_utils.to_categorical(y_data)

In [ ]:
def model_base():
  model = Sequential()
  model.add(LSTM(256, input_shape = (X.shape[1], X.shape[2]), return_sequences=True))
  model.add(Dropout(0.2))
  model.add(LSTM(256))
  model.add(Dropout(0.2))
  model.add(Dense(y.shape[1], activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
  return model

In [ ]:
model = model_base()

# Check Point

In [ ]:
file_path = 'checkpoint/weights-improvement-{epoch:02d}-{loss:02f}.hdf5'
checkpoint = ModelCheckpoint(file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

Epoch 1/50
87/87 [==============================] - 151s 2s/step - loss: 2.9868

Epoch 00001: loss improved from inf to 2.98680, saving model to checkpoint/weights-improvement-01-2.986804.hdf5
Epoch 2/50
87/87 [==============================] - 150s 2s/step - loss: 2.9535

Epoch 00002: loss improved from 2.98680 to 2.95350, saving model to checkpoint/weights-improvement-02-2.953495.hdf5
Epoch 3/50
87/87 [==============================] - 157s 2s/step - loss: 2.9435

Epoch 00003: loss improved from 2.95350 to 2.94347, saving model to checkpoint/weights-improvement-03-2.943474.hdf5
Epoch 4/50
87/87 [==============================] - 154s 2s/step - loss: 2.9294

Epoch 00004: loss improved from 2.94347 to 2.92942, saving model to checkpoint/weights-improvement-04-2.929422.hdf5
Epoch 5/50
87/87 [==============================] - 155s 2s/step - loss: 2.9128

Epoch 00005: loss improved from 2.92942 to 2.91277, saving model to checkpoint/weights-improvement-05-2.912769.hdf5
Epoch 6/50
87/87 [=

# Loada model

In [ ]:
mo = model_base()
mo.load_weights('/content/checkpoint/weights-improvement-49-0.635666.hdf5')
mo.compile(loss='categorical_crossentropy', optimizer='adam')

# Generate Text

In [ ]:
#Mapeo inverso
int_to_char = dict([(i,c) for i, c in enumerate(char_to_int)])

In [ ]:
def generate_speed(X_data):
  start = np.random.randint(0, len(X_data)-1)
  pattern  = X_data[start]
  return pattern

In [ ]:
def generate_text(model, speed, n_vocab, size):
  for i in range(size):
    x = np.reshape(speed, (1, len(speed), 1))
    x = x /float(n_vocab)
    prediction = mo.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in speed]
    sys.stdout.write(result)
    speed.append(index)
    speed = speed[1:len(speed)]
  return result 

In [ ]:
speed = generate_speed(X_data)
generate_text(mo, speed, n_vocab, 500)

techo; habían arrandado las vigas para construiriar de los gutres y había escondido en uno de sus libho de golgerta delía mesoo de campo  ne lala rua teria  ee caspo le lalpe, eue ta panda  aspiuosa  lue labía amminddado lo les pantada els prrdn; a los muehnstes del tegies iicileelleste y las suegecraia la larantó y acrió: era la muchacha. en la oscuridad  don sn escucin, euminosa  pue era librepensador por el piemor ce la slagura don uno de los tres que tañana yantaba y noniera arando la corenc

'c'